In [1]:
from scipy.io import loadmat
data = loadmat('trainingData.mat')
data = data['dataRecord']

In [2]:
data

array([[ 0.13934752, -0.13738079, -0.09019745, ..., -0.01442425,
        -0.01393293, -0.02809704],
       [ 0.14289331, -0.13554322, -0.08676443, ..., -0.01483599,
        -0.01492737, -0.02676274],
       [ 0.13994413, -0.09746756, -0.08743045, ..., -0.01704501,
        -0.01450799, -0.02508828],
       ...,
       [ 0.12818809, -0.13837552, -0.13200116, ..., -0.01336238,
        -0.01472691, -0.0283519 ],
       [ 0.10762956, -0.1349313 , -0.11258663, ..., -0.01250316,
        -0.0147819 , -0.02912535],
       [ 0.12365232, -0.14092316, -0.13252528, ..., -0.0129535 ,
        -0.01433413, -0.02912045]])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 假设你的数据是numpy数组形式，需要转换成torch Tensor
import numpy as np

# 示例数据，你应该用你的实际数据替换这里的np.random.rand()
inputs = torch.tensor(data[:, :6], dtype=torch.float32)
outputs = torch.tensor(data[:, 6:], dtype=torch.float32)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义神经网络模型
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(6, 32)  # 输入层到隐藏层，50个神经元
        self.fc2 = nn.Linear(32, 16) # 第二个隐藏层，20个神经元
        self.fc3 = nn.Linear(16, 3)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

model = SimpleFNN()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 200
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 5.9410929679870605e-05
Epoch 2, Loss: 2.4316802864632336e-06
Epoch 3, Loss: 1.2566556506499182e-06
Epoch 4, Loss: 2.291849114044453e-06
Epoch 5, Loss: 1.3247415608930169e-06
Epoch 6, Loss: 1.8292598724656273e-06
Epoch 7, Loss: 9.89362774816982e-07
Epoch 8, Loss: 1.3413274473350612e-06
Epoch 9, Loss: 1.3826330587107805e-06
Epoch 10, Loss: 1.502313921264431e-06
Epoch 11, Loss: 1.4210256722435588e-06
Epoch 12, Loss: 1.4100294265517732e-06
Epoch 13, Loss: 1.0440272717460175e-06
Epoch 14, Loss: 1.037105448631337e-06
Epoch 15, Loss: 1.8855182588595198e-06
Epoch 16, Loss: 1.3391363609116524e-06
Epoch 17, Loss: 9.024308269545145e-07
Epoch 18, Loss: 9.461496688345505e-07
Epoch 19, Loss: 8.744757451495389e-07
Epoch 20, Loss: 1.0309705658073653e-06
Epoch 21, Loss: 7.811858040440711e-07
Epoch 22, Loss: 1.5859909581195097e-06
Epoch 23, Loss: 1.3436289236778975e-06
Epoch 24, Loss: 6.492135184998915e-07
Epoch 25, Loss: 1.2173383083791123e-06
Epoch 26, Loss: 6.465365345320606e-07
Epoch 

In [4]:

# 预测
test_input = torch.tensor([[0.11412254, -0.09698317, -0.13648682, -0.08168529,  0.09584415,
        -0.08081015]], dtype=torch.float32)  # 示例输入数据
with torch.no_grad():
    prediction = model(test_input)
    print(f'Prediction: {prediction.numpy()}')
    print("-0.01443261, -0.01519005, -0.02687549")

Prediction: [[-0.01464863 -0.01573505 -0.02644522]]
-0.01443261, -0.01519005, -0.02687549


In [5]:
# save the model
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'model.pth')

In [6]:
data = loadmat('testingData.mat')
test_data = data['dataRecord']

# 测试数据集，应该与训练数据集具有相同的结构
test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_outputs = torch.tensor(test_data[:, 6:], dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_inputs, test_outputs)
test_loader = DataLoader(test_dataset, batch_size=10)

# 将模型设置为评估模式
model.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')


Average loss on the test dataset: 3.408337164501063e-07
